# Figure 1 Data: Depth vs. Exposure Time

***
### Table of Contents

1. [Information](#Information)
2. [Imports](#Imports)
3. [Data Location](#Data-Location)
4. [Generate the Data](#Generate-the-Data)
5. [Issues](#Issues)
6. [About this Notebook](#About-this-Notebook)
***

## Information

#### JDox links: 
* [NIRCam Imaging Sensitivity](https://jwst-docs.stsci.edu/display/JTI/NIRCam+Imaging+Sensitivity#NIRCamImagingSensitivity-Depthvs.exposuretime)
    * Figure 1. Depth vs. exposure time
 
*** 
<div class="alert alert-warning">

**Warning:** This notebook assumes you have installed Pandeia according to the JDox instructions here: [Installing Pandeia](https://jwst-docs.stsci.edu/display/JPP/Installing+Pandeia)

</div>

***

## Imports

In [ ]:
import os
import json
import numpy as np  
import astropy.io.fits as pyfits
import matplotlib.pyplot as plt
%matplotlib inline

from pandeia.engine.perform_calculation import perform_calculation

## Data Location

#### Background data: 

Background data are installed as part of the Pandeia installation (i.e., the files are located wherever you assigned your ```pandeia_refdata``` environment variable). See [Setting up the Pandeia Environment](https://jwst-docs.stsci.edu/display/JPP/Installing+Pandeia#InstallingPandeia-SettingupthePandeiaEnvironment) for more details.

After the data is generated, it will be kept in a Box folder here along with the associated JSON files:
[ST-INS-NIRCAM -> JDox -> nircam_imaging_sensitivity](https://stsci.box.com/s/zzlofpyysmxfc9uxaqj893hjfxeo4j7j)

#### Load the background data and JSON file

(The next cell assumes you downloaded the data into your ```Users/$(logname)/``` home directory)

In [ ]:
# BKG
bkg_level = "low" # low, medium, high

bg_file = os.environ['pandeia_refdata']+"/background/minzodi_"+bkg_level+".fits"
print('BKG: '+bg_file)

bg_table = pyfits.getdata(bg_file)
background = [bg_table['wavelength'],bg_table['background']] # wavelength in micron, SB in MJy/sr

# JSON
jsonfile = 'nircam_imaging.json'  
print('JSON: '+jsonfile)

with open(jsonfile) as f:
    imgr_data = json.load(f)

## Generate the Data

#### Create ladder of exposure times using recommended readout patterns and numbers of groups

In [ ]:
#readmodes = 'rapid bright1 bright2 shallow2 shallow4 medium2 medium8 deep2 deep8'.split()
readmodes = 'bright1 shallow4 medium8 deep8'.split()  # bright2 ngroups > 4 not allowed

readpats = []
for readmode in readmodes:
    if 'bright' in readmode:
        ngroups = np.arange(5,11)  # NGROUPS = 1 doesn't work
    elif 'shallow' in readmode:
        ngroups = np.arange(5,11)  # NGROUPS = 1 doesn't work
    elif 'medium' in readmode:
        ngroups = np.arange(6,11)  # pick up where the last group left off
    elif 'deep' in readmode:
        ngroups = np.arange(6,21)  # pick up where the last group left off
    for ngroup in ngroups:
        readpat = readmode + ' %d' % ngroup
        readpats.append(readpat)
        #print readpat

#### Perform the calculation

In [ ]:
# Test
#results = perform_calculation(imgr_data)

######
# Edit parameters and try one calculation

imgr_data['scene'][0]['spectrum']['normalization']['norm_flux'] = mag = 29
imgr_data['scene'][0]['spectrum']['normalization']['norm_fluxunit'] = 'abmag'
imgr_data['configuration']['detector']['nexp'] = nexp = 4
imgr_data['configuration']['detector']['nint'] = nint = 1
imgr_data['configuration']['detector']['ngroup'] = ngroup = 5
imgr_data['configuration']['detector']['readmode'] = readmode = 'bright1'
imgr_data['configuration']['instrument']['filter'] = filt = 'f200w'
imgr_data['background'] = background

ch = 'sw'
imgr_data['configuration']['instrument']['aperture'] = ch
imgr_data['configuration']['instrument']['mode'] = ch+'_imaging'
imgr_data['strategy']['aperture_size'] = 0.08  # radius (default 0.1")
imgr_data['strategy']['sky_annulus'] = 0.6, 0.99  # (default 0.22" - 0.4")

results = perform_calculation(imgr_data)
exptime = results['scalar']['exposure_time']

line = readmode.ljust(10)
snr = results['scalar']['sn']
line += '%2d  %2d  %2d  %7.2f  %12.8f' % (ngroup, nint, nexp, exptime, snr)
print line
print "Initial test complete..."
print

######

snr0 = 10  # desired SNR

outdir = 'results_snr%d' % snr0
if not os.path.exists(outdir):
    os.mkdir(outdir)

filts = 'f090w f115w f150w f200w f277w f356w f410m f444w'.split()
for filt in filts:
    print filt
    lam = int(filt[1:4])
    if lam < 240:
        ch = 'sw'
        imgr_data['strategy']['aperture_size'] = 0.08  # radius (default 0.1")
        imgr_data['strategy']['sky_annulus'] = 0.6, 0.99  # (default 0.22" - 0.4")
    else:
        ch = 'lw'
        imgr_data['strategy']['aperture_size'] = 0.16  # radius (default 0.1")
        imgr_data['strategy']['sky_annulus'] = 1.2, 1.98  # (default 0.22" - 0.4")

    imgr_data['configuration']['instrument']['aperture'] = ch
    imgr_data['configuration']['instrument']['mode'] = ch+'_imaging'

    imgr_data['configuration']['instrument']['filter'] = filt

    ireadpat = 0

    mags = np.arange(25.9,31,0.2) + 0.1
    mags = mags - 2.5 * np.log10(snr0 / 5.)
    for mag in mags:
        print filt, 'mag', mag
        nexp = 4
        imgr_data['scene'][0]['spectrum']['normalization']['norm_flux'] = mag
        imgr_data['configuration']['detector']['nexp'] = nexp
    
        outfile = outdir + '/etc_snr_%dexp_%s_mag%5.2f.txt' % (nexp, filt, mag)
        if os.path.exists(outfile):
            print outfile, 'EXISTS.  SKIPPING...'
            continue
        
        fout = open(outfile, 'w')
        snr = 0
        ireadpat1 = ireadpat
        while snr < snr0:
            readpat = readpats[ireadpat]
            #print filt, readpat
            readmode, ngroup = readpat.split()
            ngroup = int(ngroup)
            imgr_data['configuration']['detector']['ngroup'] = ngroup
            imgr_data['configuration']['detector']['readmode'] = readmode
            results = perform_calculation(imgr_data)
            exptime = results['scalar']['exposure_time']
            snr = results['scalar']['sn']
            line = readmode.ljust(10)
            line += '%2d  %2d  %2d  %7.2f  %12.8f' % (ngroup, nint, nexp, exptime, snr)
            print line
            if snr > snr0:
                if ireadpat:
                    if ireadpat == ireadpat1:
                        # If SNR > snr0 on the first attempt, then go back to previous shallower pattern
                        print 'Going back one...'
                        readpat = readpats[ireadpat-1]
                        readmode, ngroup = readpat.split()
                        ngroup = int(ngroup)
                        imgr_data['configuration']['detector']['ngroup'] = ngroup
                        imgr_data['configuration']['detector']['readmode'] = readmode
                        results1 = perform_calculation(imgr_data)
                        exptime1 = results1['scalar']['exposure_time']
                        snr1 = results1['scalar']['sn']
                        line1 = readmode.ljust(10)
                        line1 += '%2d  %2d  %2d  %7.2f  %12.8f' % (ngroup, nint, nexp, exptime1, snr1)
                        print line1
                        fout.write(line1+'\n')
                    
            fout.write(line+'\n')
            #print readmode, ngroup, exptime, snr
            if snr < snr0:
                ireadpat += 1
                
            if ireadpat == len(readpats):
                break  # final read pattern didn't yield snr > snr0

        fout.close()
        print outfile
        print
        
        if ireadpat == len(readpats):
            break  # mag loop; can't go fainter; final read pattern didn't yield snr > snr0

## Issues

* This runs very slowly in the notebook. Should we just keep it as a standalone script?

## About this Notebook
**Authors:**   
Dan Coe & Alicia Canipe

**Updated On:**   
April 10, 2019